In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM
from keras.layers import Dropout
from os.path import isfile, join
from keras.layers import Flatten, Conv1D, Input, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras import layers as L
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
import pickle
import os
from os import listdir
import pywt
from pywt import dwt, wavedec
from os.path import isfile, join
import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

2021-12-09 15:46:23.222187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-09 15:46:23.222217: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df,
                 batch_size, validation,
                 shuffle=True):

        self.df = df.copy()
        if validation:
            self.df = self.df[25000:]
        else:
            self.df = self.df[:25000]
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path):
    
        objects = []
        with (open(path, "rb")) as openfile:
            while True:
                try:
                    objects.append(pickle.load(openfile))
                except EOFError:
                    break
        return objects[0]["audio"]

    
    def __get_output(self, tag):
        return lab[tag]
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        files = batches["file"]
        tag = batches["tag"]

        X_batch = np.asarray([self.__get_input(x) for x in files])

        y_batch = np.asarray([self.__get_output(y) for y in tag])

        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [4]:
modelsPath = "/home/ankursharma/"

models = ["best_modeldb4level7.hdf5"]

In [5]:
import keras
# reconstructed_model = keras.models.load_model("/home/ankursharma/Outputs/best_modeldb4.hdf5")

In [6]:
all_models = []
for i in models:
    all_models.append(os.path.join(modelsPath,i))

In [ ]:
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_modeldb10.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=[mc,es], use_multiprocessing=True, workers=6)

In [ ]:
all_models

In [7]:
all_models
model = []
for i in all_models:
    model.append(keras.models.load_model(i))

2021-12-09 15:46:41.005763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-09 15:46:41.005916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-09 15:46:41.006037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-12-09 15:46:41.006158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libc

In [ ]:
dft = pd.read_csv('dataGenbior1.5level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[0].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGencoif10level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[1].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGendb4level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[2].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGenrbio2.4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[3].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGensym4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[4].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenbior1.5.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[5].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGencoif3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[6].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGendb12.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[7].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenrbio2.4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[8].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGensym6.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[9].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenbior2.4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[10].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGencoif6.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[11].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGendb4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[12].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenrbio6.8.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[13].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGensym9.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[14].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenbior3.5.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[15].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGencoif8.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[16].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenrbio1.3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[17].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGensym16.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[18].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

In [ ]:
dft = pd.read_csv('dataGenbior1.5level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[0].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGencoif10level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[1].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGendb4level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[2].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGenrbio2.4level3.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[3].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

dft = pd.read_csv('dataGenbior1.5level7.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[4].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGencoif10level7.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[5].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

# dft = pd.read_csv('dataGendb4level7.csv')
# dat = []
# files = dft["file"]
# for i in files:
#     if i.split("/")[-2] not in dat:
#         dat.append(i.split("/")[-2])

# lab = {}

# for i in range(84):
#     tag = dat[i]
#     y_label = np.zeros((84))
#     y_label[i] = 1
#     lab[tag] = y_label
    
# dft = shuffle(dft)
# training_generator = CustomDataGen(dft, 32, False)
# validation_generator = CustomDataGen(dft, 32, True)
# model[6].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)


dft = pd.read_csv('dataGenrbio2.4level7.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[6].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

In [9]:
dft = pd.read_csv('dataGendb4level7.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label
    
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
model[0].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)

/tmp/ipykernel_27708/2122823727.py:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model[0].fit_generator(generator=training_generator, validation_data = validation_generator, epochs=1, use_multiprocessing=True, workers=6)
2021-12-09 15:47:15.262913: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 2100 num_cores: 64 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 1048576 l3_cache_size: 23068672 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


781/781 [==============================] - ETA: 0s - loss: 0.1162 - accuracy: 0.9612

2021-12-09 15:49:20.420604: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 2100 num_cores: 64 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 1048576 l3_cache_size: 23068672 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


781/781 [==============================] - 141s 176ms/step - loss: 0.1162 - accuracy: 0.9612 - val_loss: 0.1053 - val_accuracy: 0.9681
